# Count number of mutual friends
- For each user having ID in the column userId, count the amount of his / her mutualfriends with each other user having ID in the column userId.

- Print 49 pairs of the users having the largest amount of mutual friends, ordered in descending order first by the common friends count , then by id of user1 and finally by id of user 2. The format is following: "count user1 user2"

In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [8]:
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array
from pyspark.sql import Window

reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .select(col("friend"),sort_array(col("users"), asc=True).alias("users")) \
    .filter(size("users") > 1) \

In [9]:
reversedGraph.printSchema()

root
 |-- friend: integer (nullable = true)
 |-- users: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, ArrayType
import itertools

def get_all_pairs(users_list):
    return [ list(el) for el in itertools.combinations(users_list,2) ]

get_all_pairs_UDF = udf(get_all_pairs, ArrayType(ArrayType(IntegerType())))

explodedGraph = reversedGraph.withColumn("user_pairs", explode(get_all_pairs_UDF("users")))

In [11]:
explodedGraph = explodedGraph.drop('users').drop('users_size') \
    .withColumn('user1', col("user_pairs")[0]) \
    .withColumn('user2', col("user_pairs")[1]) \

In [12]:
from pyspark.sql.functions import count

friends_count = explodedGraph.groupBy(["user1", "user2"]) \
    .agg(count("friend").alias("friends_count"))

In [13]:
window = Window.orderBy(col("friends_count").desc())

top50 = friends_count.withColumn("row_number", row_number().over(window)) \
    .filter(col("row_number") < 50) \
    .select(col("friends_count"), col("user1"), col("user2")) \
    .orderBy(col("friends_count").desc(),col("user1").desc(),col("user2").desc()) \
    .collect()

In [18]:
for val in top50:
    #print '%s %s %s' % (val[0],val[1],val[2])
    if val[1]<val[2]:
        print '%s %s %s' % (val[0],val[1],val[2])
    else:
        print '%s %s %s' % (val[0],val[2],val[1])

3206 27967558 42973992
3130 20158643 42973992
3066 22582764 42973992
3044 21864412 51640390
3021 17139850 51640390
3010 14985079 51640390
2970 17139850 21864412
2913 20158643 27967558
2903 22280814 51151280
2870 23848749 51640390
2855 20158643 22582764
2849 20158643 44996025
2846 22280814 42973992
2784 21864412 23848749
2779 31964081 51640390
2776 39205988 51640390
2754 17139850 23848749
2749 22582764 27967558
2728 50561859 51640390
2724 15485897 51640390
2700 28135661 42973992
2655 22280814 27967558
2653 42973992 43548989
2639 26755857 51640390
2621 14635589 51640390
2608 15485897 17139850
2606 17139850 26755857
2601 21864412 39205988
2600 8406745 51640390
2599 37735419 51640390
2597 20158643 28135661
2585 40003405 42973992
2585 21864412 31964081
2581 27967558 43548989
2579 23848749 31964081
2578 27967558 28135661
2578 15485897 21864412
2577 42973992 64755069
2574 51151280 57077210
2573 20158643 43548989
2566 21864412 26755857
2564 22280814 64755069
2561 42973992 44996025
2556 1713985